In [1]:
import insightface

In [2]:
face_model = insightface.app.FaceAnalysis()
face_model.prepare(ctx_id=-1, det_size=(640, 640))


/Users/thaivu/.pyenv/versions/3.10.2/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/thaivu/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127

In [3]:
from imutils import paths, resize
import cv2
import os

In [4]:
from tqdm import tqdm

In [5]:
id_mapping = {}
name_mapping = ""

In [6]:
DETECT_THRESHOLD = 0.6

def embedding_extract(imgs_folder, id2name):
    global id_mapping, name_mapping
    
    id_mapping = {}
    name_mapping = id2name
    count = 0

    for img_path in paths.list_images(imgs_folder):
        try:
            dossier_id = img_path.split(os.path.sep)[-2]
            print(f"[DEBG] dossier_id: {dossier_id}")
            img = cv2.imread(img_path)
            # TODO resize
            img = resize(img, width=250)
            faces = face_model.get(img)
            if len(faces) == 1:
                for face in faces:
                    if face.det_score < DETECT_THRESHOLD:
                        continue
                    embedding = face.embedding.flatten()
                    print(f"[DEBUG] embedding size: {embedding.shape}")
                    id_mapping[str(count)] = dossier_id
                    count += 1
        except Exception as e:
            continue


    print("[DEBUG] extract successful")

In [7]:
embedding_extract('lesserafim', 'lesserafim')

[DEBG] dossier_id: sakura


/Users/thaivu/.pyenv/versions/3.10.2/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


[DEBUG] embedding size: (512,)
[DEBG] dossier_id: sakura
[DEBG] dossier_id: chaewon
[DEBUG] embedding size: (512,)
[DEBG] dossier_id: eunchae
[DEBUG] embedding size: (512,)
[DEBG] dossier_id: yunjin
[DEBUG] embedding size: (512,)
[DEBG] dossier_id: kazuha
[DEBUG] embedding size: (512,)
[DEBUG] extract successful


In [8]:
print(id_mapping)
print(name_mapping)

{'0': 'sakura', '1': 'chaewon', '2': 'eunchae', '3': 'yunjin', '4': 'kazuha'}
lesserafim


In [9]:
import time
from vidgear.gears import CamGear

def analyze(video_path, sub_dir, video_id):
    """ Analyze video

    Args:
        video_path  : video full path
        sub_dir     : where to save dossier images found
        video_id    : ID of video

    Returns:
        results     : List<(dossier_id/name, image_path)>
    """
    print(f'[DEBUG] id_mapping: {id_mapping}')
    results = []
    unknown_faces = []

    milestone = 5
    extra = 5

    status = 'PROCESSING'
    # before_time = time.time()

    stream = CamGear(
        source=video_path,
        logging=True
    ).start()
    frame_rate = 60
    frame_id = 0
    start_time = time.time()
    completed_frames = 0

    try:
        while True:
            # Continue retraining from video_frame_index
            frame = stream.read()

            if frame is None:
                break

            if frame_id % int(frame_rate) == 0:
                try:
                    completed_frames += 1
                    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
                    faces = face_model.get(small_frame)
                    if len(faces) > 0:
                        timestamp = time.time() - start_time
                        print(f"[{completed_frames}] found {len(faces)} faces in {timestamp}")
                        cv2.imshow('frame', small_frame)
                except Exception as e:
                    continue
        
            key = cv2.waitKey(1) & 0xFF
            if key == ord("q"):
                break   
    finally:
        cv2.destroyAllWindows() 
        stream.stop()
        print("[INFO] finished video analysing")
    return results, status

14:31:52 ::    Helper     ::   INFO   :: Running VidGear Version: 0.2.6


In [ ]:
analyze("./short_antifragile.mp4", "dossiers", "antifragile")